# Продвинутый Python, семинар 12

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Комбинируем потоки и корутины

Зачем это нужно?

Допустим, что вы настраиваете работу системы (вспоминаем ООП из ДЗ-2). Вам что необходимо? Вы ждете заказов от пользователя, но пока вы их ждете, то у вас же все не простаивает на месте: заказы собираются, курьеры возят. И все это синхронно. Давайте осуществлять мечту

In [ ]:
import time
import asyncio


def blocking_io():
    print(f"{time.strftime('%X')} block IO")
    time.sleep(5) # снимает GIL
    print(f"{time.strftime('%X')} unblock IO")

non_block = asyncio.to_thread(blocking_io) # без скобочек!!! аргументы просто через пробел, версия 3.9+

async def main():
    print(f"{time.strftime('%X')} start async")
    _ = await asyncio.gather(non_block, asyncio.sleep(5))
    print(f"{time.strftime('%X')} finish async")

asyncio.run(main())

Аналогично с теми же самими факториалами (опять-таки, это нам не даст ускорения по сравнению с просто корутинами, потому что слишком малые рассчеты) и опять-таки - внутри Python паралеллить вычисления - дело такое себе

In [ ]:
import time
import asyncio

def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        time.sleep(0.1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f


fact_1 = asyncio.to_thread(factorial, "A", 2)
fact_2 = asyncio.to_thread(factorial, "B", 3)
fact_3 = asyncio.to_thread(factorial, "C", 4)

async def main():
    print(f"{time.time()} beg")
    res = await asyncio.gather(
        fact_1,
        fact_2,
        fact_3
    )
    print(f"{time.time()} end")

asyncio.run(main())


## А теперь к примеру использования

Сегодня мы поговорим с вами про пример асинхронности в виде все того же любимого Телеграм-бота, а в следующий раз будем говорить про асинхронность для БД (потому что как мы уже говорили, это позволяет использовать потоки)

Зачем в Телеграм боте нужна асинхронность, спросите вы? Ответ прост: это гораздо быстрее работает (особенно чем больше нагрузка, тем больше вам нужна асинхронность), ну а также это позволяет более эффективно использовать параллельность (которая, сама собой, и так есть в телеграм-боте)

В чем разница? По существу, вместо requests используется [aiohttp](https://docs.aiohttp.org/en/stable/) - асинхронная версия requests, если так можно высказаться

Сделаем сразу вещь, которую нельзя сделать без асинхронности: шедулер! В этом нам поможет бибилиотека [aioschedule](https://github.com/ibrb/python-aioschedule) - асинхронный шедулер. Работает также, как и шедулер (про который мы говорили на ООП), но только асинхронный

In [ ]:
import asyncio
import aioschedule
from telebot.async_telebot import AsyncTeleBot

API_TOKEN = '<api_token>'
bot = AsyncTeleBot(API_TOKEN)


async def beep(chat_id) -> None:
    """Send the beep message."""
    await bot.send_message(chat_id, text='Beep!')
    aioschedule.clear(chat_id)


@bot.message_handler(commands=['help', 'start'])
async def send_welcome(message):
    await bot.reply_to(message, "Hi! Use /set <seconds> to set a timer")


@bot.message_handler(commands=['set'])
async def set_timer(message):
    args = message.text.split()
    if len(args) > 1 and args[1].isdigit():
        sec = int(args[1])
        aioschedule.every(sec).seconds.do(beep, message.chat.id).tag(message.chat.id)
    else:
        await bot.reply_to(message, 'Usage: /set <seconds>')


@bot.message_handler(commands=['unset'])
def unset_timer(message):
    aioschedule.clean(message.chat.id)


async def scheduler():
    while True:
        await aioschedule.run_pending()
        await asyncio.sleep(1)


async def main():
    await asyncio.gather(bot.infinity_polling(), scheduler())


if __name__ == '__main__':
    asyncio.run(main())

Зачем еще нужна асинхронность? Опять-таки для подключения к БД. Допустим, что вы храните информацию по юзерам: как их зовут и так далее (та самая проблема, которую вы получали на ДЗ для обработки бана юзера и так далее). Так вот, для создания более сложных ботов (например, магазина), вам нужно подключаться к БД, делать асинхронного бота, и таким образом, запрос еще ждать. А остальные ждут и ждут...

В целом это и будет темой для последнего ДЗ)

## Животное дня

Итак, сегодня у нас ламы! На самом деле ламы - это 4 отдельных вида, которых стоит различать. Какие есть виды?

* Гуанако

* Викунья

* Лама

* Альпака

![](https://funart.pro/uploads/posts/2021-07/1626722411_27-funart-pro-p-vikunya-zhivotnoe-zhivotnie-krasivo-foto-30.jpg)

Это гуанако. Они дикие и прыгучие, а одомашненный вид - это лама

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Black_Llama.jpg/1920px-Black_Llama.jpg)

Викунья - дикая, меньше и стройнее, чем гуанако (различить сложно)

![](https://i.artfile.ru/2048x1365_851478_[www.ArtFile.ru].jpg)

А одомашненный вид - это альпака. Она пышная и пушистая!

![](https://static.fjcdn.com/large/pictures/6d/7d/6d7d59_5574469.jpg)

Лам используют в качестве вьючных животных (но они очень гордые, положи на них больше - они пошлют)

А альпак - для того, чтобы их стричь и из шерсти делать одежду и так далее

Одомашнили примерно 5000 тыс лет назад предками инков и прочих народов Анд